In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID column 'EIN'
application_df.drop(columns=['EIN'], inplace=True)

In [3]:
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [5]:
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [9]:
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [10]:
application_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [11]:
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [17]:
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [18]:
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [23]:
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [20]:
application_df['NAME'].value_counts().loc[lambda x: x>10]

PARENT BOOSTER USA INC                                                    1260
TOPS CLUB INC                                                              765
UNITED STATES BOWLING CONGRESS INC                                         700
WASHINGTON STATE UNIVERSITY                                                492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                            408
                                                                          ... 
USA SWIMMING INC                                                            11
SIGMA ALPHA OMEGA CHRISTIAN SORORITY INC                                    11
SOCIETY FOR PRESERVATION & ENCOURMT OF BARBERSHOP QUARTET SINGING AMER      11
SONS OF ITALY IN AMERICA                                                    11
FELLOWSHIP OF CHRISTIAN COWBOYS INC                                         11
Name: NAME, Length: 204, dtype: int64

In [6]:
# Drop the columns with minimal variance 'STATUS' and 'SPECIAL_CONSIDERATIONS'
application_df.drop(columns=['STATUS','SPECIAL_CONSIDERATIONS'], inplace=True)

In [15]:
#Define function that bins rare values as "Other"
def binning(column_name, min_val):
    value_list = application_df[column_name].unique()
    values_to_bin = []
    for value in value_list:
        if application_df[column_name].value_counts()[value] < min_val:
            values_to_bin.append(value)

    for val in values_to_bin:
        application_df[column_name] = application_df[column_name].replace(val,"Other")

    return application_df[column_name].value_counts()    

In [22]:
application_df.columns

Index(['NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [24]:
#Bin rare values for each categorical column
binning('NAME', 10)
binning('APPLICATION_TYPE', 1000)
binning('AFFILIATION', 100)
binning('CLASSIFICATION', 1000)
binning('USE_CASE', 300)
binning('ORGANIZATION', 500)

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [31]:
#Convert catergorical features to numeric
app_df_dummies = pd.get_dummies(application_df)

In [32]:
y= app_df_dummies['IS_SUCCESSFUL'].values
X= app_df_dummies.drop(columns=['IS_SUCCESSFUL']).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21)

In [33]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
#hidden_nodes_layer3 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features, 
                             activation='tanh'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,  
                             activation='relu'))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3,  
#                             activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               25600     
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,701
Trainable params: 30,701
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [45]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4708 - accuracy: 0.7681
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4481 - accuracy: 0.7809
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4444 - accuracy: 0.7811
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4417 - accuracy: 0.7832
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4413 - accuracy: 0.7837
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4392 - accuracy: 0.7841
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4384 - accuracy: 0.7847
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4382 - accuracy: 0.7855
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4369 - accuracy: 0.7866
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4359 - accura

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4574 - accuracy: 0.7911 - 353ms/epoch - 1ms/step
Loss: 0.45743465423583984, Accuracy: 0.7911370396614075


In [47]:
# Export model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

reg = LogisticRegression(random_state=21)
reg.fit(X_train_scaled, y_train)
y_pred = reg.predict(X_test_scaled)

print(f"Logistic Regression Accuracy: {accuracy_score(y_test,y_pred)}")

Logistic Regression Accuracy: 0.7821574344023323
